# Convert a series of calexp into fits files from sources Light Curve

- author Sylvie Dagoret-Campagne
- creation date 2024-05-29
- last update 2024-05-31
- affiliation : IJCLab
- Kernel **w_2024_16**


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm


import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits
from astropy.wcs import WCS


import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 100)

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
import pickle
from collections import OrderedDict

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'x-large'
plt.rcParams['axes.titlesize'] = 'x-large'
plt.rcParams['xtick.labelsize']= 'x-large'
plt.rcParams['ytick.labelsize']= 'x-large'

In [ ]:
import gc

In [ ]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time


In [ ]:
import lsst.daf.butler as dafButler
#import lsst.summit.utils.butlerUtils as butlerUtils

In [ ]:
import lsst.afw.image as afwImage
import lsst.afw.display as afwDisplay
import lsst.afw.table as afwTable
import lsst.geom as geom

In [ ]:
# LSST Display
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [ ]:
transform = AsinhStretch() + PercentileInterval(99.)

### Example to fit a gaussian with astropy

- https://github.com/sylvielsstfr/Fors2pcigale/blob/19-plot-fitted-emission-lines-from-notebooks/src/gelato/Example/ExampleFitInNb.ipynb

In [ ]:
from astropy.modeling import models, fitting
from astropy import modeling
# define a model for a line
g_init = models.Gaussian1D(amplitude=1, mean=0, stddev=1)
# initialize a linear fitter
fit_g = fitting.LevMarLSQFitter()

In [ ]:
m = modeling.models.Gaussian1D(amplitude=10, mean=30, stddev=5)
x = np.linspace(0, 100, 2000)
data = m(x)
data = data + np.sqrt(data) * np.random.random(x.size) - 0.5
data -= data.min()
plt.plot(x, data)

In [ ]:
fitter = modeling.fitting.LevMarLSQFitter()
model = modeling.models.Gaussian1D()   # depending on the data you need to give some initial values
fitted_model = fit_g(model, x, data)

In [ ]:
fitted_model.mean.value

Import statements that we will need later

Let's make a new plot and metric tool, we'll base it on the example in the getting started guide.

In [ ]:
# INSERT YOUR collection and tract
# for rehearsal use collection 2 which have CCDvisit
butlerRoot = "/repo/embargo"

collection1 = 'LSSTComCamSim/runs/nightlyvalidation/20240402/d_2024_03_29/DM-43612'
collection2 = 'LSSTComCamSim/runs/nightlyvalidation/20240403/d_2024_03_29/DM-43612'
collection3 = 'LSSTComCamSim/runs/nightlyvalidation/20240404/d_2024_03_29/DM-43612'
#collection = 'LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240311/w_2024_10/PREOPS-4985'
collection = 'LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240414/w_2024_15/PREOPS-5069' # COMPLETED

collectionn = collection
#collections = [collection1,collection2,collection3]
collections = [collection]
collectionStr = collectionn.replace("/", "_")
fn_ccdVisit_tracts_patches = f"ccdVisittractpatch_{collectionStr}.csv"
instrument = 'LATISS'
skymapName = "latiss_v1"
where_clause = "instrument = \'" + instrument+ "\'"
tract = 3864 # mostly for light-curves
patch_sel = 236
band = 'g'
#tract = 5615
# tract = 5634 # interesting to view calib parameters
suptitle = collectionStr + f" inst = {instrument} tract = {tract}"

In [ ]:
#dataId = {"skymap": "latiss_v1", "tract": 5615, "instrument": "LATISS"}
dataId = {"skymap": skymapName, "tract": tract, "instrument": instrument}
repo = '/sdf/group/rubin/repo/oga/'
butler = dafButler.Butler(repo)
#t = Butler.get(table_sel, dataId=dataId, collections=collections)
registry = butler.registry

In [ ]:
skymap_auxtel ='latiss_v1'
skymap = butler.get('skyMap', skymap=skymap_auxtel, collections=collections)

In [ ]:
def remove_figure(fig):
    """
    Remove a figure to reduce memory footprint.

    Parameters
    ----------
    fig: matplotlib.figure.Figure
        Figure to be removed.

    Returns
    -------
    None
    """
    # get the axes and clear their images
    for ax in fig.get_axes():
        for im in ax.get_images():
            im.remove()
    fig.clf()       # clear the figure
    plt.close(fig)  # close the figure
    gc.collect()    # call the garbage collector

In [ ]:
def get_corners_radec(wcs, bbox):
    """
    Return the corners in RA,Dec in degrees given the WCS and bounding box for an image.

    Parameters
    ----------
    wcs: image WCS returned by the Butler
    bbox: bounding box returned by the Butler

    Returns
    -------
    corners_ra, corners_dec in decimal degrees
    """

    corners_x = [bbox.beginX, bbox.beginX, bbox.endX, bbox.endX]
    corners_y = [bbox.beginY, bbox.endY, bbox.endY, bbox.beginY]
    corners_ra = []
    corners_dec = []
    for i in range(4):
        radec = wcs.pixelToSky(corners_x[i], corners_y[i])
        corners_ra.append(radec.getRa().asDegrees())
        corners_dec.append(radec.getDec().asDegrees())
    
    return corners_ra, corners_dec

In [ ]:
def convert_fluxtomag(x) :
    """
    The object and source catalogs store only fluxes. There are hundreds of flux-related columns, 
    and to store them also as magnitudes would be redundant, and a waste of space.
    All flux units are nanojanskys. The AB Magnitudes Wikipedia page provides a concise resource 
    for users unfamiliar with AB magnitudes and jansky fluxes. To convert to AB magnitudes use:
    As demonstrated in Section 2.3.2, to add columns of magnitudes after retrieving columns of flux, users can do this:
    results_table['r_calibMag'] = -2.50 * numpy.log10(results_table['r_calibFlux']) + 31.4
    results_table['r_cModelMag'] = -2.50 * numpy.log10(results_table['r_cModelFlux']) + 31.4
    (from DP0 tutorial)
    """
    return -2.50 * np.log10(x) + 31.4

## Get Pixel Scale

In [ ]:
import lsst.geom as geom
import lsst.sphgeom

skymap = butler.get('skyMap', skymap=skymapName, collections=collections )
tractInfo = skymap.generateTract(tract)
for patch in tractInfo:    
    patchID = patch.getSequentialIndex()
        
    ibb=patch.getInnerBBox()
    tWCS=tractInfo.getWcs()
       
    # loop on the 4 corners
    for icorn,corner in enumerate(ibb.getCorners()):
        p = geom.Point2D(corner.getX(), corner.getY())
        coord = tWCS.pixelToSky(p)

In [ ]:
tWCS

In [ ]:
#arcsec/pixel
pixel_scale = tWCS.getPixelScale().asArcseconds()

## Selected visits

In [ ]:
inputfilename = "sources_objectTable-t3864-bg-o547-LATISS_runs_AUXTEL_DRP_IMAGING_20230509_20240414_w_2024_15_PREOPS-5069.csv"
#inputfilename = "sources_objectTable-t3864-bg-o912-LATISS_runs_AUXTEL_DRP_IMAGING_20230509_20240414_w_2024_15_PREOPS-5069.csv"
df_myselectedvisits = pd.read_csv(inputfilename ,index_col=0)
tract = 3864
band = "g"
objectname = 547
title = f"Auxtel Light Curves : tract = {tract}, band = {band}, object = {objectname} "
suptitle = inputfilename 

In [ ]:
df_myselectedvisits.sort_values("visit",inplace=True)
#df_myselectedvisits.sort_index(inplace=True)

In [ ]:
df_myselectedvisits

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(14,4))
df_myselectedvisits.plot.scatter(x="expMidptMjd",y="psfMag",ax=ax,s=20,c="zeroPoint",cmap="jet",grid=True,rot=45)
ax.set_title(title)
plt.suptitle(suptitle)
plt.tight_layout()
plt.show()

In [ ]:
df_myselectedvisits["visit"]

Note here:
- zeroPoint is in ABMag
- skyBg is in ADU (but don't know if in pixel or in arcsec2)
- skyNoise is in ADU (but don't know if in pixel or in arcsec2)

In [ ]:
index = 3
visitId = df_myselectedvisits.iloc[index]['visit']

In [ ]:
row_source = df_myselectedvisits.iloc[index]
row_source

In [ ]:
x_target = row_source['x']
y_target = row_source['y']
ra_target= row_source['ra']
dec_target= row_source['dec']

In [ ]:
datasetType = 'calexp'
dataId = {'visit': visitId, 'instrument':instrument , 'detector': 0}
datasetRefs = registry.queryDatasets(datasetType, dataId=dataId, collections  = collections)

for i, ref in enumerate(datasetRefs):
    print(ref.dataId)
    print("band:", ref.dataId['band'])
    band = ref.dataId['band']

## One Calexp 

In [ ]:
calexp = butler.get('calexp', **dataId,collections=collections)
mask = calexp.mask.array

In [ ]:
the_mask = np.where(mask==0,1,0)
the_mask.shape

In [ ]:
im = plt.imshow(the_mask,origin="lower",cmap="grey")
plt.colorbar(im)

In [ ]:
img = calexp.image.array * (the_mask)

In [ ]:
im=plt.imshow(img,origin="lower",vmin=-10,vmax=10,cmap="grey")
plt.colorbar(im)

In [ ]:
%matplotlib widget

calexp = butler.get('calexp', **dataId,collections=collections)

calexp_info = calexp.getInfo()
photocalib = calexp_info.getPhotoCalib()
expo_photocalibconstant_mean = photocalib.getCalibrationMean()
expo_photocalibconstant_error = photocalib.getCalibrationErr()
calexp_md = calexp_info.getMetadata()
magzero,magzero_rms,magzero_nobj = calexp_md["MAGZERO"],calexp_md["MAGZERO_RMS"],calexp_md["MAGZERO_NOBJ"] 
bgmean,bgvar = calexp_md["BGMEAN"],calexp_md["BGVAR"]

mask = calexp.mask.array
the_mask = np.where(mask==0,1,0)
#img = calexp.maskedImage.getImage()
img = calexp.image.array 
#img = calexp[the_mask] 
wcs = calexp.getWcs()
#bbox = img.getBBox()
#corners_ra, corners_dec = get_corners_radec(wcs, bbox)
#extent = [np.min(corners_ra),np.max(corners_ra),np.min(corners_dec),np.max(corners_dec)]


# Display the image with a suitable scaling
data_flat = img.flatten()
mean = np.mean(data_flat)
med = np.median(data_flat)
sigMad = 1.4826 * np.median(np.fabs(data_flat - mean))
#vmin = med - 3 * sigMad
#vmax = med + 3 * sigMad
vmin = med - 50.
vmax = med + 50.
the_min= data_flat.min()
the_max= data_flat.max()
print(mean,med ,sigMad ,vmin ,vmax,the_min,the_max )

fig,axs = plt.subplots(1,2,figsize=(16,8))

## Plot the image
ax = axs[0]
from astropy.visualization import ZScaleInterval
z = ZScaleInterval()
z1,z2 = z.get_limits(img)
from matplotlib.colors import SymLogNorm
im=ax.imshow(img, origin="lower", cmap="grey",norm=SymLogNorm(linthresh=0.1))
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
the_title = f"Calexp : visit = {visitId}, band = {band}"
ax.set_title(the_title)
ax.grid()
ax.set_aspect('equal')
ax.scatter([x_target],[y_target],marker ="o", s= 100,facecolors='none', edgecolors='r',lw=1)
plt.colorbar(im, cax=cax,ax=ax)



ax = axs[1]

#val = data_flat/med)
#data_flat_mag = -2.5*np.log10((data_flat/med))*1000
#mu_mag = np.mean(data_flat_mag)
#med_mag = np.median(data_flat_mag)
#sigMad_mag = 1.4826 * np.median(np.fabs(data_flat_mag - med_mag))
#vmin_mag = med_mag - 3 * sigMad_mag
#vmax_mag = med_mag + 3 * sigMad_mag
#textstr = '\n'.join((
#    r'$\mu=%.2f$ mmag' % (mu_mag, ),
#    r'$\mathrm{med}=%.2f$ mmag' % (med_mag, ),
#    r'$\sigma=%.2f$ mmag' % (sigMad_mag , )))

data_flat_nozero = data_flat[~(data_flat==0)]
histdata  = ax.hist(data_flat_nozero,bins=100,range=(vmin,vmax),histtype="step",color="b",lw=2)
histarray,histedges = histdata[0],histdata[1]
ax.set_ylim(0.,histarray.max()*1.1)

# fit
g = fit_g(g_init,histedges[1:],histarray)
print("fig_g parameters: ",g)
m = modeling.models.Gaussian1D(amplitude=g.amplitude.value, mean=g.mean.value, stddev=g.stddev.value)
ax.plot(histedges,m(histedges),"r-")
textstr = '\n'.join((
    r'$\mu=%.2f$ ADU/pixel' % (g.mean.value, ),
    r'$\sigma=%.2f$ ADU/pixel' % (g.stddev.value , ),
    r'$\sigma=%.2f$ mag/s/arcsec2' % (convert_fluxtomag(g.stddev.value*expo_photocalibconstant_mean/30./pixel_scale**2) ,)))
props = dict(boxstyle='round', facecolor='wheat' ,alpha=0.5)
ax.text(0.05, 0.95, textstr, color="r",transform=ax.transAxes, fontsize=10,verticalalignment='top', bbox=props)
ax.set_title("Calexp Sky-bkg residuals")
ax.set_xlabel("Flux (ADU)")

#ax.grid()

plt.suptitle(collections)
plt.tight_layout()
plt.show()
#remove_figure(fig)

### Need to make the WCS object info fits file

https://keflavich-astropy.readthedocs.io/en/latest/wcs/


- how to convert DM wcs to astropy wcs :
- https://github.com/rubin-dp0/tutorial-notebooks/blob/main/DP02_03a_Image_Display_and_Manipulation.ipynb

In [ ]:
# Convert WCS-DM to astropy WCS
the_fits_WCS = WCS(calexp.getWcs().getFitsMetadata())

In [ ]:
the_fits_WCS

In [ ]:
the_fits_WCS.to_header_string()

In [ ]:
# create the header
header = the_fits_WCS.to_header()

In [ ]:
header

In [ ]:
md = calexp_md.toDict() 
arr= img
# 180 degree rotation
rotated_array = arr[::-1,::-1] #rotate the array 180 degrees
filename_out = f"exposure_{visitId}_calexp.fits"

In [ ]:
for key,value in md.items():
# DS9 cannot read long cards    
#    if len(key)<19:
    if len(key)<8:
        header[str(key)]=value

In [ ]:
primary_hdu = fits.PrimaryHDU(header=header)
#image_hdu = fits.ImageHDU(rotated_array)
image_hdu = fits.ImageHDU(arr)
hdu_list = fits.HDUList([primary_hdu, image_hdu])
hdu_list.writeto(filename_out,overwrite=True)    

## Check the wcs from rubin

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = plt.subplot(projection=the_fits_WCS)
#plt.subplot(projection=the_fits_WCS)
calexp_extent = (calexp.getBBox().beginX, calexp.getBBox().endX,
                 calexp.getBBox().beginY, calexp.getBBox().endY)
im = ax.imshow(calexp.image.array, cmap='gray', vmin=vmin, vmax=vmax,extent=calexp_extent, origin='lower')
plt.grid(color='white', ls='solid')
plt.xlabel('Right Ascension')
plt.ylabel('Declination')
ax.scatter([ra_target],[dec_target],marker ="o", s= 100,facecolors='none', edgecolors='r',transform=ax.get_transform('icrs'),lw=1)

the_title = f"Calexp : visit = {visitId}, band = {band}"
plt.title(the_title)
plt.show()
#remove_figure(fig)

## Check by reading the fits file with its WCS

In [ ]:
hdulist = fits.open(filename_out)

# Parse the WCS keywords in the primary HDU
w = WCS(hdulist[0].header)
data = hdulist[1].data

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = plt.subplot(projection=w)
im = ax.imshow(data, cmap='gray', vmin=vmin, vmax=vmax, origin='lower',aspect='equal')
ax.grid(color='white', ls='solid')
ax.set_xlabel('Right Ascension')
ax.set_ylabel('Declination')
ax.scatter([ra_target],[dec_target],marker ="o", s= 100,facecolors='none', edgecolors='r',transform=ax.get_transform('icrs'),lw=1)
overlay = ax.get_coords_overlay('icrs')
overlay.grid(color='white', ls='dotted')

the_title = f"Calexp : visit = {visitId}, band = {band}"
plt.suptitle(the_title,fontsize=20)
plt.show()